# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""

In [4]:
# 取票價(Fare), 對乘客年齡(Age)做群聚編碼
df['Fare'] = df['Fare'].fillna('None')
df['Age'] = df['Age'].fillna(df['Age'].mean())

mean_df = df.groupby(['Fare'])['Age'].mean().reset_index()
mode_df = df.groupby(['Fare'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Fare'])['Age'].median().reset_index()
max_df = df.groupby(['Fare'])['Age'].max().reset_index()
min_df = df.groupby(['Fare'])['Age'].min().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Fare'])
temp = pd.merge(temp, median_df, how='left', on=['Fare'])
temp = pd.merge(temp, max_df, how='left', on=['Fare'])
temp = pd.merge(temp, min_df, how='left', on=['Fare'])
temp.columns = ['Fare', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min']
temp.head()

,Fare,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,0.0000,32.239529,29.699118,29.699118,49.0,19.0
1,4.0125,20.000000,20.000000,20.000000,20.0,20.0
2,5.0000,33.000000,33.000000,33.000000,33.0,33.0
3,6.2375,61.000000,61.000000,61.000000,61.0,61.0
4,6.4375,34.500000,34.500000,34.500000,34.5,34.5


In [6]:
df = pd.merge(df, temp, how='left', on=['Fare'])
df = df.drop(['Fare'] , axis=1)
df.head()

,Pclass,Age,SibSp,Parch,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,3,22.0,1,0,29.199796,22.000000,25.000000,59.0,20.5
1,1,38.0,1,0,38.000000,38.000000,38.000000,38.0,38.0
2,3,26.0,0,0,28.555556,32.000000,27.500000,44.0,17.0
3,1,35.0,1,0,30.200000,19.000000,33.000000,37.0,19.0
4,3,35.0,0,0,30.378782,29.699118,29.699118,55.5,16.0


In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min']



,Pclass,Age,SibSp,Parch,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,3,22.0,1,0,29.199796,22.000000,25.000000,59.0,20.5
1,1,38.0,1,0,38.000000,38.000000,38.000000,38.0,38.0
2,3,26.0,0,0,28.555556,32.000000,27.500000,44.0,17.0
3,1,35.0,1,0,30.200000,19.000000,33.000000,37.0,19.0
4,3,35.0,0,0,30.378782,29.699118,29.699118,55.5,16.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [ ]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

In [ ]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

In [10]:
# 沒有這些新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min'] , axis=1)

from sklearn.linear_model import LinearRegression
# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.13850161011185344

In [11]:
# 新特徵 + 線性迴歸 : 有些為改善

train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.14240083784605725

In [12]:
# 原始特徵 + 梯度提升樹
from sklearn.ensemble import GradientBoostingRegressor

train_X = MMEncoder.fit_transform(df_minus)
estimator = GradientBoostingRegressor()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

C:\ProgramData\Anaconda3\envs\venv36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.15802798481051394

In [13]:
# 新特徵 + 梯度提升樹
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.16175712800280118